In [ ]:
import pandas as pd

In [ ]:

class FPDataCapture:
    def __init__(self, base_file_path):
        self.headers_f_1 = {}
        self.headers_f_2 = {}
        self.sample_frequency = 1200
        self.base_file_path = base_file_path
        self.data_f_1 = self.import_data(self.base_file_path.replace(".tsv", "_f_1.tsv"), self.headers_f_1)
        self.data_f_2 = self.import_data(self.base_file_path.replace(".tsv", "_f_2.tsv"), self.headers_f_2)
        self.foot_lift_times = None
        self.foot_down_times = None

    def import_data(self, file_path, headers_dict):
        # The number of initial lines containing metadata information.
        num_metadata_lines = 26 
        
        # Assuming that the first line of actual data has the correct column headers
        data = pd.read_csv(file_path, delimiter='\t', header=num_metadata_lines)
        
        # Convert data to numeric, handling non-numeric entries
        data = data.apply(pd.to_numeric, errors='coerce')
        
        # Change TIME to time
        data.rename(columns={"TIME": "time"}, inplace=True)

        # Reset the index and column names of the dataframe
        data.reset_index(drop=False, inplace=True)
        column_names = list(data.columns)[1:]
        data.drop(data.columns[-1], axis=1, inplace=True)
        data.columns = column_names
        
        test_file_path = "/Users/danielcopeland/Library/Mobile Documents/com~apple~CloudDocs/MIT Masters/DRL/LABx/RADARTreePose/data/csvs/"
         
        # Save the DataFrame to a CSV file
        csv_file_path = test_file_path + str(file_path.split('/')[-1].replace('.tsv', '.csv'))
        data.to_csv(csv_file_path, index=False)
        print(f"Data saved to {csv_file_path}")

        return data

    def identify_foot_lift(self):
        # Determine which force plate data to use based on the filename content
        if "MNTRL" in self.base_file_path:
            data = self.data_f_1
        elif "MNTRR" in self.base_file_path:
            data = self.data_f_2
        else:
            raise ValueError("Filename must contain 'MNTRL' or 'MNTRR'")
        
        # Convert 'time' column to numeric for comparison
        data['time'] = pd.to_numeric(data['time'], errors='coerce')

        # Identifying foot lift and put down events
        # A foot lift event is identified when COP_X and COP_Y go from non-zero to zero
        foot_lift_events = data[(data['COP_X'].shift(1) != 0) & (data['COP_Y'].shift(1) != 0) &
                                (data['COP_X'] == 0) & (data['COP_Y'] == 0)]

        # A foot down event is identified when COP_X and COP_Y go from zero to non-zero
        foot_down_events = data[(data['COP_X'].shift(1) == 0) & (data['COP_Y'].shift(1) == 0) &
                                (data['COP_X'] != 0) & (data['COP_Y'] != 0)]

        # Filter out foot lift events that are too close to each other (within 0.1 seconds)
        filtered_lift_times = []
        for t in data.loc[foot_lift_events.index, 'time']:
            if not filtered_lift_times or t - filtered_lift_times[-1] > 0.1:
                filtered_lift_times.append(t)

        # Filter out foot down events that are too close to foot lift events
        filtered_down_times = []
        for t in data.loc[foot_down_events.index, 'time']:
            if not any(abs(t - lift_time) <= 0.1 for lift_time in filtered_lift_times):
                filtered_down_times.append(t)

        # Now filter foot down events that are too close to each other (within 0.1 seconds)
        final_filtered_down_times = []
        for t in filtered_down_times:
            if not final_filtered_down_times or t - final_filtered_down_times[-1] > 0.1:
                final_filtered_down_times.append(t)

        # Save the times for foot lift and put down events
        self.foot_lift_times = filtered_lift_times
        self.foot_down_times = final_filtered_down_times

        # Returning the times where foot lift and put down events occur
        return self.foot_lift_times, self.foot_down_times

# Usage example:
# file_path_f_1 = 'path_to_f_1.tsv' # Replace with actual file path
# file_path_f_2 = 'path_to_f_2.tsv' # Replace with actual file path
# fp_data_capture = FPDataCapture(file_path_f_1, file_path_f_2)
# filename_to_check = 'some_filename_containing_MNTRL_or_MNTRR'
# fp_data_capture.identify_foot_lift(filename_to_check)
# Now you can access the times directly
# print(fp_data_capture.foot_lift_times_f_1)
# print(fp_data_capture.foot_down_times_f_1)
# print(fp_data_capture.foot_lift_times_f_2)
# print(fp_data_capture.foot_down_times_f_2)
